# Improved ResNet

In [2]:
import tensorflow as tf
print("GPUs Available:", len(tf.config.list_physical_devices('GPU')))


GPUs Available: 1


In [3]:
# Imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import cv2
import os
import sys
from io import StringIO
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import tensorflow.keras.applications.resnet50 as resnet
import warnings
warnings.filterwarnings('ignore')

In [10]:
!pip install iterative-stratification

In [4]:
print(os.listdir("/kaggle/input"))

# Path to competition dataset
data_dir = "/kaggle/input/grand-xray-slam-division-b"
# Check what files are inside
print('Filenames of the data', os.listdir(data_dir))

['grand-xray-slam-division-b']
Filenames of the data ['test2', 'sample_submission_2.csv', 'train2.csv', 'train2']


In [5]:
# Load the training CSV metadata with labels
train = pd.read_csv("/kaggle/input/grand-xray-slam-division-b/train2.csv")

print('Metadata shape:',train.shape)
train.head()

Metadata shape: (108494, 21)


,Image_name,Patient_ID,Study,Sex,Age,ViewCategory,ViewPosition,Atelectasis,Cardiomegaly,Consolidation,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,00000003_001_001.jpg,3,1,Male,41.0,Frontal,AP,0,1,0,...,1,0,0,1,0,0,0,0,0,0
1,00000004_001_001.jpg,4,1,Female,20.0,Frontal,PA,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,00000004_001_002.jpg,4,1,Female,20.0,Lateral,Lateral,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,00000006_001_001.jpg,6,1,Female,42.0,Frontal,AP,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,00000010_001_001.jpg,10,1,Female,50.0,Frontal,PA,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [6]:
# 1. Feature & Target Preperation
# Define labels
conditions = [
    'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
    'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
    'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices'
]
# Features you want
features = ["ViewCategory", "ViewPosition", "Age", "Sex"]

# Encode categorical features
from sklearn.preprocessing import LabelEncoder

train_enc = train.copy()   # train data encoded
for col in ["ViewCategory", "ViewPosition", "Sex"]:  # features that can be encoded
    le = LabelEncoder()
    train_enc[col] = le.fit_transform(train[col].astype(str))

X = train_enc[features].values
y = train[conditions].values
print(X.shape) # 4 features (ViewCategory, ViewPosition, Age, Sex)
print(y.shape)  # 14 conditions

(108494, 4)
(108494, 14)


In [7]:
# 2. Adding ViewBalancing for Stratification: ViewCategory= Frontal, Lateral; since ViewCategory is unbalanced

# One-hot encode ViewCategory and append to 
view_onehot = pd.get_dummies(train["ViewCategory"], prefix="view").values

y_aug = np.hstack([y, view_onehot])  # augmented target matrix (added ViewCategory as y to stratify and reduce bias)

In [11]:
# ============================================================
# ⚡ GPU-Optimized GRAND X-RAY SLAM — ResNet50 AUC Pipeline
# ============================================================
import os, gc
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts
from tensorflow.keras import mixed_precision
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# Enable GPU optimizations
tf.config.optimizer.set_jit(True)
tf.config.experimental.enable_tensor_float_32_execution(True)
mixed_precision.set_global_policy('mixed_float16')
print("✅ Mixed precision + XLA enabled for faster GPU performance.")

# ============================================================
# 🪄 Dataset Preprocessing (tf.data pipeline)
# ============================================================
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def decode_image(filename, label=None, augment=False):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.keras.applications.resnet50.preprocess_input(img)

    if augment:
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_brightness(img, 0.1)
    return (img, label) if label is not None else img

def make_dataset(df, is_test=False, augment=False):
    img_paths = ("/kaggle/input/grand-xray-slam-division-b/test2/" + df["Image_name"]) if is_test else ("/kaggle/input/grand-xray-slam-division-b/train2/" + df["Image_name"])
    labels = None if is_test else df[conditions].values.astype(np.float32)
    
    ds = tf.data.Dataset.from_tensor_slices((img_paths, labels) if not is_test else img_paths)
    if not is_test:
        ds = ds.map(lambda x, y: decode_image(x, y, augment), num_parallel_calls=tf.data.AUTOTUNE)
    else:
        ds = ds.map(lambda x: decode_image(x, None, False), num_parallel_calls=tf.data.AUTOTUNE)
    
    return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
print('**********Building ResNet Model******************')
# ============================================================
# 🧱 Build ResNet50 model
# ============================================================
def build_resnet_model(num_classes=14, unfreeze_layers=None):
    base = ResNet50(weights="imagenet", include_top=False, input_shape=(224,224,3))
    if unfreeze_layers:
        for layer in base.layers[-unfreeze_layers:]:
            layer.trainable = True
    else:
        base.trainable = False

    x = tf.keras.layers.GlobalAveragePooling2D()(base.output)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    out = tf.keras.layers.Dense(num_classes, activation="sigmoid")(x)
    return tf.keras.Model(inputs=base.input, outputs=out)

# ============================================================
# ⚙️ Callbacks
# ============================================================
def get_callbacks(fold):
    return [
        ModelCheckpoint(f"fold_{fold}_best.h5", monitor="val_AUC", mode="max", save_best_only=True, verbose=1),
        EarlyStopping(monitor="val_AUC", mode="max", patience=5, restore_best_weights=True, verbose=1),
    ]

# ============================================================
# 🚀 Cross-validation training loop
# ============================================================
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_aucs = []
print('**********Starting MSKF (k-fold)******************')

for fold, (train_idx, val_idx) in enumerate(mskf.split(X, y_aug)):
    print(f"\n================ FOLD {fold+1} ================")
    train_df = train.iloc[train_idx].reset_index(drop=True)
    val_df   = train.iloc[val_idx].reset_index(drop=True)

    # tf.data datasets instead of Sequence
    train_ds = make_dataset(train_df, augment=True)
    val_ds   = make_dataset(val_df)

    # Learning rate schedule
    lr_schedule = CosineDecayRestarts(
        initial_learning_rate=1e-4,
        first_decay_steps=len(train_ds)*2,
        t_mul=2.0, m_mul=0.9, alpha=1e-6
    )

    # Stage 1 — Train top layers
    model = build_resnet_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule),
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.AUC(name="auc")])
    history = model.fit(train_ds, validation_data=val_ds, epochs=3, callbacks=get_callbacks(fold), verbose=1)
    
    # Stage 2 — Fine-tune last 20 layers
    for layer in model.layers[-20:]: layer.trainable = True
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.AUC(name="auc")])
    history_ft = model.fit(train_ds, validation_data=val_ds, epochs=3, callbacks=get_callbacks(fold), verbose=1)

    # Save final model per fold
    model.save(f"fold_{fold}_final.h5")
    best_auc = max(history.history['val_auc'] + history_ft.history['val_AUC'])
    fold_aucs.append(best_auc)
    print(f"✅ Fold {fold+1} Best AUC: {best_auc:.4f}")
    gc.collect()
print('*************************MSKF K-FOld COMPLETE*************')
# ============================================================
# 🧾 CV Summary
# ============================================================
print(f"\n📊 Cross-validation AUCs: {fold_aucs}")
print(f"🏆 Mean CV AUC: {np.mean(fold_aucs):.4f}")

# ============================================================
# 🔮 Ensemble predictions (mean of 5 folds)
# ============================================================
print("\nGenerating final ensemble predictions...")
test_ds = make_dataset(test_df, is_test=True)
fold_preds = []

for fold in range(5):
    m = tf.keras.models.load_model(f"fold_{fold}_best.h5", compile=False)
    fold_preds.append(m.predict(test_ds, verbose=1))

final_preds = np.mean(fold_preds, axis=0)
submission = pd.DataFrame(final_preds, columns=conditions)
submission.insert(0, "Image_name", test_df["Image_name"].values)
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv created successfully!")


✅ Mixed precision + XLA enabled for faster GPU performance.
**********Building ResNet Model******************
**********Starting MSKF (k-fold)******************

================ FOLD 1 ================


I0000 00:00:1759726400.210403      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/3


I0000 00:00:1759726420.906564      87 service.cc:148] XLA service 0x7fab88012de0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759726420.907315      87 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1759726423.249802      87 cuda_dnn.cc:529] Loaded cuDNN version 90300


   3/2713 ━━━━━━━━━━━━━━━━━━━━ 2:08 48ms/step - auc: 0.4855 - loss: 1.0437   

I0000 00:00:1759726428.359804      87 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1035/2713 ━━━━━━━━━━━━━━━━━━━━ 16:12 580ms/step - auc: 0.7704 - loss: 0.5591

InvalidArgumentError: Graph execution error:

Detected at node DecodeJpeg defined at (most recent call last):
<stack traces unavailable>
Detected at node DecodeJpeg defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  Input is empty.
	 [[{{node DecodeJpeg}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_2]]
  (1) INVALID_ARGUMENT:  Input is empty.
	 [[{{node DecodeJpeg}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_multi_step_on_iterator_13164]